In [7]:
!pip install transformers[torch]

In [8]:
!pip install transformers


**Preprocessing**

In [ ]:
import nltk
import re
from nltk.stem import WordNetLemmatizer
# Download the stopwords dataset

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer




# Preprocessing function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation
    text = re.sub(r'[^\w\s.]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # # Removing stopwords
    # stop_words = set(nltk.corpus.stopwords.words('english'))
    # tokens = nltk.word_tokenize(text)
    # text = ' '.join([token for token in tokens if token not in stop_words])
    # Remove extra spaces
    text = re.sub(' +', ' ', text)


    return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


**LexRankSummarizer**

    Implementation using Sumy Library

In [ ]:
!pip install sumy
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 16.8 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=eaff602b334497fa782bf25d38775a75adb432816b1eb6b8f3a78d3e274701c2
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=777fb9dd42bb95cf0eea974abbc4998275794da1f90aae7be540f865890363dd
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt


**Generating Extractive Summary Using **Laxrank** **


In [12]:
import os

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def lexrank_summarize(text, num_sentences=2):
    summarizer = LexRankSummarizer()
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summary = summarizer(parser.document, num_sentences)
    summary_sentences = [sentence._text for sentence in summary]
    return ' '.join(summary_sentences)



def process_files(input_folder, output_folder, num_sentences):
    # Ensure the output folder exists, create if not
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each .txt file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_summary.txt")

            with open(input_path, 'r', encoding='utf-8') as file:
                input_text = file.read()
            preprocces_input=preprocess_text(input_text)
            # Apply LexRank summarization
            summary_result = lexrank_summarize(preprocces_input, num_sentences)

            # Save the summary to the output file
            with open(output_path, 'w', encoding='utf-8') as output_file:
                output_file.write(summary_result)
                print ("summary Ganerated")

if __name__ == "__main__":
    input_folder_path = "/content/drive/MyDrive/check"
    output_folder_path = "/content/drive/MyDrive/checklexrankSummary"
    num_sentences_for_summary =2

    process_files(input_folder_path, output_folder_path, num_sentences_for_summary)


summary Ganerated


**Loading Pretrained Bart-Model**

In [9]:
    # Load BART base model and tokenizer
    import pandas as pd
    from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
    import os
    import torch

# Load pre-trained BART model
    model_name = "facebook/bart-base"  # Use a smaller model
    model = BartForConditionalGeneration.from_pretrained(model_name)
    tokenizer = BartTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

**Generating Summary Using BART**

In [10]:


def bart_summarize_chunked(text, model, tokenizer, max_chunk_length=512, max_length=150, min_length=50, length_penalty=2.0, num_beams=4):
    chunks = [text[i:i+max_chunk_length] for i in range(0, len(text), max_chunk_length)]
    summary_chunks = []

    for chunk in chunks:
        inputs = tokenizer.encode(  chunk, return_tensors="pt", max_length=512, truncation=True).to(device)
        summary_ids = model.generate(inputs.to(device), max_length=max_length, min_length=min_length, length_penalty=length_penalty, num_beams=num_beams)
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summary_chunks.append(summary)

    return summary_chunks

def process_files(input_folder, output_folder, model, tokenizer, device):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}_summary.txt")

            with open(input_path, 'r', encoding='utf-8') as file:
                input_text = file.read()

            # Apply BART summarization in chunks on GPU
            summary_result = bart_summarize_chunked(input_text, model, tokenizer)

            # Print filename and summary
            print(f"File: {filename}")
            print(f"Summary: {summary_result}")

            # Save the summary to the output file
            with open(output_path, 'w', encoding='utf-8') as output_file:
                output_file.write(summary_result)

if __name__ == "__main__":
    input_folder_path = "/content/drive/MyDrive/checklexrankSummary"
    output_folder_path = "/content/drive/MyDrive/Finall_summary"

    # Load BART base model and tokenizer on GPU
    model_name = "facebook/bart-base"  # Use a smaller model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    process_files(input_folder_path, output_folder_path, model, tokenizer, device)


In [ ]:
import pandas as pd
from io import StringIO

# Your CSV-like text data
csv_data = "/content/drive/MyDrive/results.csv"

# Create a pandas DataFrame from the CSV-like text
df = pd.read_csv(csv_data)

# Group by 'ROUGE-Type' and calculate the average for each ROUGE type
averages = df.groupby('ROUGE-Type').mean().reset_index()

# Print the resulting DataFrame
print(averages)

                           ROUGE-Type    Task Name  Avg_Recall  Avg_Precision  \
0    ROUGE-1+StopWordRemoval+Stemming  32183.82716    0.280908       0.138726   
1    ROUGE-2+StopWordRemoval+Stemming  32183.82716    0.070023       0.024110   
2    ROUGE-L+StopWordRemoval+Stemming  32183.82716    0.376602       0.226474   
3  ROUGE-SU4+StopWordRemoval+Stemming  32183.82716    0.130407       0.041195   

   Avg_F-Score  Num Reference Summaries  
0     0.174293                      3.5  
1     0.033116                      3.5  
2     0.274089                      3.5  
3     0.058449                      3.5  


<ipython-input-2-f5681ed02233>:11: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  averages = df.groupby('ROUGE-Type').mean().reset_index()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
